In [133]:
import datetime
import ipywidgets as widgets
import requests
from bs4 import BeautifulSoup
import json
import os
import pathlib
import time

In [134]:
from_date = datetime.date.today()
search_url='https://magic.wizards.com/en/news/archive?search&page=1&category=all&author=all&order=newest'
max_pages = 100
link_file = pathlib.Path('./data/linklist.json')
check_existing = True

In [135]:
known_link_ids = []
if os.path.exists(link_file):
    with open(link_file) as f:
        link_infos = json.load(f)
        for linfo in link_infos:
            known_link_ids.append(linfo['id'])

In [136]:
widgets.DatePicker(
    description = 'Start Date',
    value = from_date
)

DatePicker(value=datetime.date(2023, 2, 7), description='Start Date', step=1)

In [137]:
def link_exists(id: str, link_ids: list, check_for_exist: bool):
    if check_for_exist:
        print(f'Checking Id {id}')
        for link_info in link_ids:
            lid = link_info['id']
            
            if id == lid:
                print(f'Link Id {id} is already in list... Will skip')
                return True
            else:
                return False

In [138]:
i = 1
search_response = requests.get(search_url)
if (search_response.status_code != 200):
    raise requests.HTTPError(search_response)

page_list = []  

while(search_response.status_code == 200 and i <= max_pages):
    time.sleep(2)
    search_response = requests.get(f'https://magic.wizards.com/en/news/archive?search&page={i}&category=all&author=all&order=newest')
    page_result = search_response.text
    soup = BeautifulSoup(page_result, 'html.parser')
    entry_list = soup.find_all(class_="css-9f4rq")
    for entry in entry_list:
        parent = entry.findParent()
        link_id = parent.get('href')
        if not link_id in known_link_ids:
            link_info = {'id': link_id,
                        'link': 'https://magic.wizards.com'+link_id,
                        'name': parent.text,
                        'date_added': datetime.datetime.now().strftime("%Y-%m-%d, %H:%M:%S")
                        }
            
            page_list.append(link_info)
            # print(f'LinkId {link_id} added')
        else:
            print(f'Link with Id {link_id} already exists')
            
        if i % 10 == 0:
            print(f'Page: {i}')
            print(search_response.status_code)
        i+=1


with open(link_file, 'w') as j:
    json.dump(page_list, j, indent=4)





Page: 10
200
Page: 20
200
Page: 30
200
Page: 40
200
Page: 50
200
Page: 60
200
Page: 70
200
Page: 80
200
Page: 90
200
Page: 100
200
